## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

In [2]:
# Import and read the charity_data.csv
raw_data_df = pd.read_csv('resources/charity_data.csv')
raw_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
alphabet_df = raw_data_df.drop(['EIN', 'NAME'], axis=1)
alphabet_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column
print(alphabet_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
alphabet_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cut_off = 500

application_types_to_replace = alphabet_df['APPLICATION_TYPE'].value_counts()[alphabet_df['APPLICATION_TYPE'].value_counts() < cut_off].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    alphabet_df['APPLICATION_TYPE'] = alphabet_df['APPLICATION_TYPE'].replace(app, 'Other')

# Check to make sure binning was successful
alphabet_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
alphabet_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2150        1
C1728        1
C2190        1
C4120        1
C3700        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
alphabet_df['CLASSIFICATION'].value_counts()[alphabet_df['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C4100        6
C2400        6
C1720        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1246        2
C3200        2
C1267        2
C1234        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cut_off = 500

classifications_to_replace = alphabet_df['CLASSIFICATION'].value_counts()[alphabet_df['CLASSIFICATION'].value_counts() < cut_off].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    alphabet_df['CLASSIFICATION'] = alphabet_df['CLASSIFICATION'].replace(cls, 'Other')
    
# Check to make sure binning was successful
alphabet_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Out of curiosity, how many applications have a status of zero?
alphabet_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [11]:
# Discard all applications with status zero
alphabet_df = alphabet_df[alphabet_df['STATUS'] == 1]
alphabet_df.shape

(34294, 10)

In [12]:
# Determine which data points are outside of the 1.5*IQR range
ask_amt = alphabet_df['ASK_AMT']

quartiles = np.quantile(ask_amt,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)

# Drop the outliers
potential_outliers = (ask_amt <= upper_bound) | (ask_amt >= lower_bound)
alphabet_df = alphabet_df.loc[potential_outliers,:]
alphabet_df.shape

(34294, 10)

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_df = pd.get_dummies(alphabet_df)
app_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
y = app_dummies_df['IS_SUCCESSFUL'].values
X = app_dummies_df.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## First Attempt

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

nn_input_dim = X_train.shape[1]

nn_units = nn_input_dim*2

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units, activation='relu', input_dim=nn_input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units/2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units/4, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 88)                3960      
_________________________________________________________________
dense_1 (Dense)              (None, 44)                3916      
_________________________________________________________________
dense_2 (Dense)              (None, 22)                990       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 23        
Total params: 8,889
Trainable params: 8,889
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=25)

Train on 25720 samples
Epoch 1/25
25720/25720 [==============================] - 4s 141us/sample - loss: 0.5668 - accuracy: 0.7229
Epoch 2/25
25720/25720 [==============================] - 2s 94us/sample - loss: 0.5542 - accuracy: 0.7310
Epoch 3/25
25720/25720 [==============================] - 2s 95us/sample - loss: 0.5516 - accuracy: 0.7314
Epoch 4/25
25720/25720 [==============================] - 2s 96us/sample - loss: 0.5501 - accuracy: 0.7329
Epoch 5/25
25720/25720 [==============================] - 2s 95us/sample - loss: 0.5482 - accuracy: 0.7333
Epoch 6/25
25720/25720 [==============================] - 2s 96us/sample - loss: 0.5475 - accuracy: 0.7332
Epoch 7/25
25720/25720 [==============================] - 2s 97us/sample - loss: 0.5463 - accuracy: 0.7336
Epoch 8/25
25720/25720 [==============================] - 3s 101us/sample - loss: 0.5452 - accuracy: 0.7336
Epoch 9/25
25720/25720 [==============================] - 2s 97us/sample - loss: 0.5451 - accuracy: 0.7337
Epoch 10/25


In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

8574/1 - 1s - loss: 0.5122 - accuracy: 0.7293
Loss: 0.550873848839687, Accuracy: 0.7292978763580322


## Second Attempt

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

nn_input_dim = X_train.shape[1]

nn_units = nn_input_dim*3

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units, activation='relu', input_dim=nn_input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units/4, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nn_units/3, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 132)               5940      
_________________________________________________________________
dense_5 (Dense)              (None, 33)                4389      
_________________________________________________________________
dense_6 (Dense)              (None, 44)                1496      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 45        
Total params: 11,870
Trainable params: 11,870
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=75)

Train on 25720 samples
Epoch 1/75
25720/25720 [==============================] - 4s 138us/sample - loss: 0.5645 - accuracy: 0.7233
Epoch 2/75
25720/25720 [==============================] - 3s 101us/sample - loss: 0.5545 - accuracy: 0.7296
Epoch 3/75
25720/25720 [==============================] - 3s 100us/sample - loss: 0.5516 - accuracy: 0.7302
Epoch 4/75
25720/25720 [==============================] - 3s 100us/sample - loss: 0.5507 - accuracy: 0.7308
Epoch 5/75
25720/25720 [==============================] - 3s 100us/sample - loss: 0.5490 - accuracy: 0.7319
Epoch 6/75
25720/25720 [==============================] - 3s 105us/sample - loss: 0.5479 - accuracy: 0.7330
Epoch 7/75
25720/25720 [==============================] - 3s 102us/sample - loss: 0.5474 - accuracy: 0.7330
Epoch 8/75
25720/25720 [==============================] - 3s 101us/sample - loss: 0.5469 - accuracy: 0.7336
Epoch 9/75
25720/25720 [==============================] - 3s 102us/sample - loss: 0.5456 - accuracy: 0.7335
Epoch

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

8574/1 - 1s - loss: 0.5301 - accuracy: 0.7282
Loss: 0.5571552399213091, Accuracy: 0.7282481789588928


## Third Attempt

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

nn_input_dim = X_train.shape[1]

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='tanh', input_dim=nn_input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 10)                450       
_________________________________________________________________
dense_9 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_10 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 681
Trainable params: 681
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=125)

Train on 25720 samples
Epoch 1/125
25720/25720 [==============================] - 3s 116us/sample - loss: 0.5843 - accuracy: 0.7102
Epoch 2/125
25720/25720 [==============================] - 2s 79us/sample - loss: 0.5594 - accuracy: 0.7291
Epoch 3/125
25720/25720 [==============================] - 2s 79us/sample - loss: 0.5550 - accuracy: 0.7308
Epoch 4/125
25720/25720 [==============================] - 2s 79us/sample - loss: 0.5526 - accuracy: 0.7305
Epoch 5/125
25720/25720 [==============================] - 2s 79us/sample - loss: 0.5511 - accuracy: 0.7315
Epoch 6/125
25720/25720 [==============================] - 2s 80us/sample - loss: 0.5500 - accuracy: 0.7326
Epoch 7/125
25720/25720 [==============================] - 2s 80us/sample - loss: 0.5491 - accuracy: 0.7320
Epoch 8/125
25720/25720 [==============================] - 2s 82us/sample - loss: 0.5483 - accuracy: 0.7320
Epoch 9/125
25720/25720 [==============================] - 2s 81us/sample - loss: 0.5479 - accuracy: 0.7328
Epoc

25720/25720 [==============================] - 2s 65us/sample - loss: 0.5405 - accuracy: 0.7380
Epoch 77/125
25720/25720 [==============================] - 2s 65us/sample - loss: 0.5404 - accuracy: 0.7374
Epoch 78/125
25720/25720 [==============================] - 2s 66us/sample - loss: 0.5402 - accuracy: 0.7366
Epoch 79/125
25720/25720 [==============================] - 2s 66us/sample - loss: 0.5399 - accuracy: 0.7368
Epoch 80/125
25720/25720 [==============================] - 2s 72us/sample - loss: 0.5400 - accuracy: 0.7371
Epoch 81/125
25720/25720 [==============================] - 2s 85us/sample - loss: 0.5403 - accuracy: 0.7378
Epoch 82/125
25720/25720 [==============================] - 2s 73us/sample - loss: 0.5400 - accuracy: 0.7380
Epoch 83/125
25720/25720 [==============================] - 2s 74us/sample - loss: 0.5398 - accuracy: 0.7383
Epoch 84/125
25720/25720 [==============================] - 2s 73us/sample - loss: 0.5399 - accuracy: 0.7382
Epoch 85/125
25720/25720 [======

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

8574/1 - 0s - loss: 0.5188 - accuracy: 0.7279
Loss: 0.5530013852751224, Accuracy: 0.7278982996940613


## Optimize the Model

In [28]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=nn_input_dim))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return nn_model

In [29]:
# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


In [30]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 61 Complete [00h 01m 02s]
val_accuracy: 0.7264987230300903

Best val_accuracy So Far: 0.7290646433830261
Total elapsed time: 00h 19m 52s
INFO:tensorflow:Oracle triggered exit


In [31]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 9,
 'num_layers': 6,
 'units_0': 7,
 'units_1': 1,
 'units_2': 9,
 'units_3': 5,
 'units_4': 3,
 'units_5': 7,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': 'd4af4c8ff13ed000dd9f08fc3a2589e9'}

In [32]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

8574/1 - 1s - loss: 0.5527 - accuracy: 0.7291
Loss: 0.5822589742239188, Accuracy: 0.7290646433830261


In [33]:
# Save and export results to an HDF5 file
best_model.save('AlphabetSoupCharity_Optimization.h5')